<a href="https://colab.research.google.com/github/avkaz/DeepLearningPetIdentification/blob/baseline/baseline_exper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## 1st -  Download utility.py file from github repository
## 2nd - Imports all functions from utility.py

import requests

# Correct raw URL for the utility.py file
url = "https://raw.githubusercontent.com/avkaz/DeepLearningPetIdentification/main/utility.py"

# Fetch and save the file locally
response = requests.get(url)
with open("utility.py", "wb") as f:
    f.write(response.content)


import utility
print("utility.py downloaded successfully.")


utility.py downloaded successfully.


In [2]:
data = utility.get_data()

In [3]:
trypets = list(data.items())[:5]

In [4]:
trypets

[('tanyny-chomutov-2024-12-21',
  {'Jméno': 'Tanyny',
   'Pohlaví': 'Samec',
   'Kraj': 'Ústecký',
   'Okres': 'Chomutov',
   'Plemeno': 'Kříženec',
   'Věk': '5 let',
   'Barva': 'Černá',
   'Velikost': 'Střední - 10-17kg',
   'url': 'https://www.psidetektiv.cz/zvire/tanyny-chomutov-2024-12-21',
   'images': ['https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190390.jpg',
    'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190391.jpg',
    'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190392.jpg',
    'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190393.jpg',
    'https://www.psidetektiv.cz/data/catalog/big/2024/12/22/img190394.jpg']}),
 ('haily-tachov-2024-12-21',
  {'Jméno': 'Haily',
   'Pohlaví': 'Samice',
   'Kraj': 'Plzeňský',
   'Okres': 'Tachov',
   'Plemeno': 'Jezevčík',
   'Věk': '5 let',
   'Barva': 'Tmavý divočák',
   'Velikost': 'Střední - 10-17kg',
   'url': 'https://www.psidetektiv.cz/zvire/haily-tachov-2024-12-21',
   'images

In [5]:
import tensorflow as tf

from tensorflow.keras.applications import EfficientNetB0


model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

In [16]:
import requests
import json
import numpy as np
import tensorflow as tf
from PIL import Image
from io import BytesIO
import pandas as pd


from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import faiss


In [17]:
def get_embedding_from_url(image_url, model):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    image = image.resize((224, 224))
    img_array = np.array(image)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    embedding = model.predict(img_array)
    return embedding

# embeddings
pet_names = []
embeddings = []

for pet_name, pet_info in trypets:
    image_url = pet_info['images'][0]
    embedding = get_embedding_from_url(image_url, model)
    pet_names.append(pet_name)
    embeddings.append(embedding)


embeddings = np.vstack(embeddings)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


In [18]:
def get_embedding_from_url(image_url, model):
    try:
        response = requests.get(image_url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image = image.resize((224, 224))
        img_array = np.array(image)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        embedding = model.predict(img_array)
        return embedding
    except (requests.RequestException, Image.UnidentifiedImageError) as e:
        print(f"Error processing image {image_url}: {e}")
        return None


In [19]:

pet_names = []
embeddings = []
skipped_images = []

for pet_name, pet_info in trypets:
    images = pet_info['images']
    for image_url in images:
        embedding = get_embedding_from_url(image_url, model)
        if embedding is not None:
            pet_names.append(pet_name)
            embeddings.append(embedding)
        else:
            skipped_images.append(image_url)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


In [20]:
embeddings = np.vstack(embeddings)

In [21]:
d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)
faiss.normalize_L2(embeddings)
index.add(embeddings)
print("FAISS index initialized and embeddings added.")

FAISS index initialized and embeddings added.


In [22]:
#similarity metrics
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

def dot_product(v1, v2):
    return np.dot(v1, v2)

def euclidean_distance(v1, v2):
    return np.linalg.norm(v1 - v2)

def manhattan_distance(v1, v2):
    return np.sum(np.abs(v1 - v2))

def l2_squared_distance(v1, v2):
    return np.sum((v1 - v2) ** 2)

# test image
egy_pet = [pet for pet in trypets if pet[0] == "egy-karlovy-vary-2024-12-17"][0]
egy_image_url = egy_pet[1]["images"][0]  # Select the first image
egy_embedding = get_embedding_from_url(egy_image_url, model)

# embedd from all except egy1
db_pet_names = []
db_image_urls = []
db_embeddings = []

for pet_name, pet_info in trypets:
    for image_url in pet_info["images"]:
        if pet_name == "egy-karlovy-vary-2024-12-17" and image_url == egy_image_url:
            continue  # Exclude the selected Egy image
        embedding = get_embedding_from_url(image_url, model)
        if embedding is not None:
            db_pet_names.append(pet_name)
            db_image_urls.append(image_url)
            db_embeddings.append(embedding)

#sim metrics
results = []
for pet_name, image_url, embedding in zip(db_pet_names, db_image_urls, db_embeddings):
    similarity_cosine = cosine_similarity(egy_embedding[0], embedding[0])
    similarity_dot = dot_product(egy_embedding[0], embedding[0])
    distance_euclidean = euclidean_distance(egy_embedding[0], embedding[0])
    distance_manhattan = manhattan_distance(egy_embedding[0], embedding[0])
    distance_l2_squared = l2_squared_distance(egy_embedding[0], embedding[0])
    results.append((pet_name, similarity_cosine, similarity_dot, distance_euclidean, distance_manhattan, distance_l2_squared))


df = pd.DataFrame(
    results,
    columns=["Dog Name", "Cosine Similarity", "Dot Product", "Euclidean Distance", "Manhattan Distance", "L2-Squared Distance"]
)


print(df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
                          Dog Name  Cosine Similarity  Dot Product  \
0       tanyny-chomutov-2024-12-21           0.161092    20.851746   
1       tanyny-chomutov-2024-12-21           0.159638    19.857014   
2       tanyny-chomutov-2024-12-21           0.241349    30.299604   
3       tanyny-chomutov-2024-12-21           0.246545    33.722416   
4       tanyny-chomutov-2024-12-21           0.155641    21.519821   
5      